In [1]:
import math
import string
import time
from collections import defaultdict
from helpful_functions import *
import re
import copy
import functools

In [2]:
with open("day20_input.txt", "r") as f:
    prog_inp = f.readlines()

prog_inp = [line.rstrip() for line in prog_inp]

for i in range(len(prog_inp)-1,-1,-1):
    if prog_inp[i] == '': del prog_inp[i]
    else: break

print(prog_inp[:5])

['Tile 2729:', '.....#####', '##..##.#.#', '......#.##', '#....#.#.#']


In [3]:
tiles = []
index_to_id = dict()
for i, tile in enumerate(group_by(prog_inp, '')):
    index_to_id[i] = int(tile[0].split(' ')[-1][:-1])
    tiles.append([list(l) for l in tile[1:]])

tiles[0]

[['.', '.', '.', '.', '.', '#', '#', '#', '#', '#'],
 ['#', '#', '.', '.', '#', '#', '.', '#', '.', '#'],
 ['.', '.', '.', '.', '.', '.', '#', '.', '#', '#'],
 ['#', '.', '.', '.', '.', '#', '.', '#', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '#', '#', '.'],
 ['#', '.', '.', '.', '#', '.', '.', '#', '.', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '#', '#', '.'],
 ['#', '.', '#', '.', '.', '#', '.', '.', '.', '.'],
 ['#', '.', '#', '.', '#', '#', '#', '.', '.', '#']]

In [4]:
# in the format: {side : (index, side_index, is_flipped)}
# side index is 0 for top, 1 for right, 2 for bottom, and 3 for left
side_to_index = defaultdict(lambda: set())

def get_index_from_side(side):
    global side_to_index
    return side_to_index[tuple(side)]

def add_index_for_side(side, side_index, index):
    global side_to_index
    side_to_index[tuple(side)].add((index, side_index, False))
    side_to_index[tuple(side[::-1])].add((index, side_index, True))

def get_tile_sides(tile):
    return [
        tile[0],
        [l[-1] for l in tile],
        tile[-1][::-1],
        [l[0] for l in tile][::-1]
    ]

def add_index_for_tile(tile, index):
    for i, side in enumerate(get_tile_sides(tile)):
        add_index_for_side(side, i, index)

for i, tile in enumerate(tiles):
    add_index_for_tile(tile, i)

In [5]:
potential_corners = []
for i, tile in enumerate(tiles):
    nr_shared_sides = 0
    for side in get_tile_sides(tile):
        if len(get_index_from_side(side)) == 2:
            nr_shared_sides += 1
    if nr_shared_sides == 2:
        potential_corners.append((tile, i))

print("Number (potential) corners found:", len(potential_corners))
res = 1
for corner in potential_corners:
    res *= index_to_id[corner[1]]
print("multiplication of corner ids:", res)

Number (potential) corners found: 4
multiplication of corner ids: 2699020245973


In [6]:
def print_tile(tile):
    for row in tile:
        print("".join(row))

In [7]:
def rotate_tile(tile, n=1):
    n = n % 4
    if n == 0: return copy.deepcopy(tile)
    new_tile = []
    for i in range(len(tile[0])-1, -1, -1):
        new_tile.append([])
        for j in range(len(tile)):
            new_tile[-1].append(tile[j][i])
    if n > 1:
        return rotate_tile(new_tile, n - 1)
    return new_tile

# Flip over the x axis
def yflip_tile(tile):
    return tile[::-1]

# Flip over the y axis
def xflip_tile(tile):
    return [l[::-1] for l in tile]

test = [[0,1,2,'a'],
        [3,4,5,'b'],
        [6,7,8,'c']]

# r = rotate_tile(test, 3)
# r = xflip_tile(test)
# for l in r: print(l)

In [8]:
top_left_corner = None
for corner in potential_corners:
    sides = get_tile_sides(corner[0])
    if len(get_index_from_side(sides[0])) == 1:
        if len(get_index_from_side(sides[3])) == 1:
            top_left_corner = corner

In [9]:
def get_matching_tile(tile, tile_index, side_index):
    tile_side = get_tile_sides(tile)[side_index]
    matching_tiles = get_index_from_side(tile_side)

    if len(matching_tiles) == 1:
        return False
    elif len(matching_tiles) > 2:
        print("too many matches")
    
    nr_matches = 0
    for matching_tile_data in matching_tiles:
        if matching_tile_data[0] != tile_index:
            nr_matches += 1
            matching_tile_index = matching_tile_data[0]
            matching_tile = tiles[matching_tile_data[0]]
            matching_tile = rotate_tile(matching_tile, - side_index + matching_tile_data[1] + 2)
            # Winding order makes it so the side needs to be flipped if not already
            if not matching_tile_data[2]:
                if side_index % 2 == 0:
                    matching_tile = xflip_tile(matching_tile)
                else:
                    matching_tile = yflip_tile(matching_tile)

    if nr_matches != 1:
        print("wrong number of matches:", nr_matches)
    if tile_side[::-1] != get_tile_sides(matching_tile)[(side_index+2)%4]:
        print("tiles don't match")

    return (matching_tile, matching_tile_index)

tiled_picture = [[top_left_corner]]
for row_i in range(100):
    for col_i in range(100):
        last_tile = tiled_picture[-1][-1]
        next_tile = get_matching_tile(last_tile[0], last_tile[1], 1)
        if next_tile == False:
            break
        tiled_picture[-1].append(next_tile)
        # break
    # break
    
    last_tile = tiled_picture[-1][0]
    next_tile = get_matching_tile(last_tile[0], last_tile[1], 2)
    if next_tile == False:
        break
    tiled_picture.append([next_tile])

picture = []
for tile_row in tiled_picture:
    for i in range(len(tile_row[0][0])):
        if i == 0 or i == len(tile_row[0][0])-1: continue
        picture.append([])
        for j in range(len(tile_row)):
            picture[-1] += tile_row[j][0][i][1:-1]
    # # Uncomment to add split characters between the tiles
    #         picture[-1].append(' '+str(tile_row[j][1])+' ')
    # picture.append([' '])

picture = ["".join(l) for l in picture]
# for line in picture:
#     print(line)

In [10]:
sea_monster = """
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
"""[1:-1].split('\n')

def find_sea_monsters(picture):
    x_size = len(sea_monster[0])
    y_size = len(sea_monster)
    for xi in range(len(picture[0]) - x_size + 1):
        for yi in range(len(picture) - y_size + 1):
            matches = True
            coordinates = []
            for rxi in range(x_size):
                for ryi in range(y_size):
                    if sea_monster[ryi][rxi] == '#':
                        texel = picture[ryi+yi][rxi+xi]
                        if texel == '#' or texel == 'O':
                            coordinates.append((rxi+xi, ryi+yi))
                        else:
                            matches = False
                            break
                if matches == False:
                    break
            if matches:
                for coord in coordinates:
                    picture[coord[1]][coord[0]] = 'O'

trans_pic = copy.deepcopy(picture)
for n in range(4):
    trans_pic = rotate_tile(trans_pic, 1)
    find_sea_monsters(trans_pic)
    trans_pic = yflip_tile(trans_pic)
    find_sea_monsters(trans_pic)
    trans_pic = yflip_tile(trans_pic)

choppiness = 0
for line in trans_pic:
    choppiness += line.count('#')

print("sea choppiness:", choppiness)

sea choppiness: 2012
